In [ ]:
import streamlit as st
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer

# Load dataset
csv_file = "C:/Users/malli/Downloads/large_multilingual_translation_dataset.csv"
df = pd.read_csv(csv_file)

# Available translation models
language_pairs = {
    "English to French": "Helsinki-NLP/opus-mt-en-fr",
    "English to Spanish": "Helsinki-NLP/opus-mt-en-es",
    "English to German": "Helsinki-NLP/opus-mt-en-de",
    "French to English": "Helsinki-NLP/opus-mt-fr-en",
    "Spanish to English": "Helsinki-NLP/opus-mt-es-en",
    "German to English": "Helsinki-NLP/opus-mt-de-en"
}

# Streamlit App Interface
st.title("Real-Time Language Translation (NMT)")
st.write("Bridging Language Barriers with Neural Machine Translation")

# User Input
selected_pair = st.selectbox("Select Language Pair", list(language_pairs.keys()))
user_text = st.text_area("Enter text to translate:")

if st.button("Translate") and user_text:
    source_lang, target_lang = selected_pair.split(" to ")
    
    # Check dataset for translation
    if source_lang in df.columns and target_lang in df.columns:
        match = df[df[source_lang].str.lower() == user_text.lower()]
        if not match.empty:
            translated_text = match[target_lang].values[0]
            st.subheader("Translated Text (From Dataset):")
            st.success(translated_text)
        else:
            st.write("Translation not found in dataset. Using NMT model...")
            use_model = True
    else:
        use_model = True

    # Use model if dataset lookup fails
    if 'use_model' in locals() and use_model:
        model_name = language_pairs[selected_pair]
        tokenizer = MarianTokenizer.from_pretrained(model_name)
        model = MarianMTModel.from_pretrained(model_name)
        
        # Tokenize and Translate
        inputs = tokenizer([user_text], return_tensors="pt", padding=True, truncation=True)
        translated = model.generate(**inputs)
        translated_text = tokenizer.batch_decode(translated, skip_special_tokens=True)[0]
        
        # Display Output
        st.subheader("Translated Text (Using Model):")
        st.success(translated_text)

# Footer
st.markdown("---")
st.write("Developed with ❤ using Hugging Face Transformers, Streamlit & Dataset")